Funtion in python are a first class objects. Now what is a first class object?

A program entity is a first class object if it can be:
- Created at runtime
- Assigned to a variable or element in a data structure
- Passed as argument to a function 
- returned as argument to a function

In [1]:
# to demonstrate this
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n*factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'returns n!'

In [4]:
type(factorial)

function

In [5]:
# demonstrace the other factors that make functions a first-class obj
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [7]:
map(factorial, range(11))

In [8]:
list(map(factorial, range(5)))

[1, 1, 2, 6, 24]

## Higher-Order Functions

Having first-class functions enables programming in a functional style. One of the hall-marks of functional programming is the use of Higher-order functions

A function that takes another function as argument or returns a function is called a higher order function. This can be seen in the `sorted` function which has a keyword argumnet `key` which takes a function.

In [9]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [10]:
# any one-argument function can be used as the key
def reverse(word):
    return word[::-1]

reverse('testing')

'gnitset'

In [11]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

`map`, `filter` and `reduce` are the best-know higher order functions. But there are better alternatives to these and we will see them below.

`listcomps` and `genexps` replace the functionalities of `map` and `filter` and in most cases they are more readable.

In [12]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [13]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [14]:
list(map(factorial, filter(lambda n: n%2, range(6))))

[1, 6, 120]

In [15]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

In python3 `map` and `filter` return generators so their direct substituion is `genexps`.

In [17]:
# the reduce fun is replaced by built-in sum fun which is more fast and readable
from functools import reduce
from operator import add
reduce(add, range(100))

4950

In [18]:
sum(range(100))

4950

## Anonymous Functions

using the `lambda` keyword, you can create a anonymous function within a python expression. the `lambda` function cannot be used to create complex functions with `while` `try` etc. but its best use is in the context of arguments lists.

for example lets rewrite the reverse function we saw earlier using lambda.

In [1]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## User-defined Callable types

not only are Python functions real object, arbitary Python objects may also be made to behave like functions by implementing a `__call__` instance method.

In [6]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

In [13]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [14]:
bingo()

0

## Function Introspection

function, as we mentioned earlier are first-class objects. Now lets look at the factors that make its so.
```
>> dir(factorial)
```
**The `__dict__` attribute**: Like instances of plain user-defined class, a function uses `__dict__` attribute to store user attributes assigned to it.

Looking closer at attributes that are specific to functions. To see the detailed explaination of each attribute ref Table-5.1 pg 148.

In [16]:
class C: pass
obj = C()

def func(): pass

sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

In [17]:
# python has an extremely flexible parameter handling mechanism
def tag(name, *content, cls=None, **attrs):
    """Generates one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
        
    if attrs:
        attr_str = ''.join('%s="%s"' % (attr, value)
                          for attr, value
                          in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [18]:
tag('br')

'<br />'

In [19]:
tag('p', 'hello')

'<p>hello</p>'

In [20]:
tag('p', 'hello', 'world')

'<p>hello</p>\n<p>world</p>'

In [21]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
'src': 'sunset.jpg', 'cls': 'framed'}

In [22]:
tag(**my_tag)

'<imgclass="framed"src="sunset.jpg"title="Sunset Boulevard" />'